## 1.Load libs

In [1]:
%pip install scikit-image scikit-learn seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
import cv2
import numpy as np
import pathlib
import os

2024-03-24 16:16:47.383502: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 16:16:47.414162: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 16:16:47.414207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 16:16:47.415121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 16:16:47.420131: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 16:16:47.420683: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
from sklearn.model_selection import cross_validate

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, log_loss

In [6]:
import matplotlib.pyplot as plt

In [7]:
import seaborn as sns

## 2. Create model


In [8]:
layersOfInterest = ["conv1_conv", 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']

In [9]:
layers_of_models = {}

In [10]:
def _loadLayer(_nameLayer=None):
    global layers_of_models
    
    _model = tf.keras.applications.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None, classes=1000)
    
    output = _model.get_layer(_nameLayer).output
    
    output = tf.keras.layers.GlobalAveragePooling2D()(output)
    
    model_selected = tf.keras.Model(_model.input, output)
    
    layers_of_models[_nameLayer] = model_selected
    
    return model_selected

In [11]:
def cross_validation(model, _X, _y, _cv=5):
    _scoring = ['accuracy']
    results = cross_validate(estimator=model,
                            X=_X,
                            y=_y,
                            cv=_cv,
                            scoring=_scoring,
                            return_train_score=True)

    return {"Training Accuracy scores": results['train_accuracy'],
            "Mean Training Accuracy": results['train_accuracy'].mean()*100,
            "Validation Accuracy scores": results['test_accuracy'],
            "Mean Validation Accuracy": results['test_accuracy'].mean()*100}

In [12]:
for layer in layersOfInterest:
    _loadLayer(layer)
    
    if(layers_of_models[layer]):    
        print(f'- Load of model: {layer} [Finished]')
    else:
        print(f'- Load of model: {layer} [Failed]')

2024-03-24 16:16:49.127312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 16:16:49.128053: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


- Load of model: conv1_conv [Finished]
- Load of model: conv2_block3_out [Finished]
- Load of model: conv3_block4_out [Finished]
- Load of model: conv4_block6_out [Finished]
- Load of model: conv5_block3_out [Finished]


Visualization of all models loaded in object 

In [13]:
tf.keras.applications.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None, classes=1000).summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [14]:
layers_of_models

{'conv1_conv': <keras.src.engine.functional.Functional at 0x7b98ba4d8e50>,
 'conv2_block3_out': <keras.src.engine.functional.Functional at 0x7b98b8037bd0>,
 'conv3_block4_out': <keras.src.engine.functional.Functional at 0x7b98b23f9b10>,
 'conv4_block6_out': <keras.src.engine.functional.Functional at 0x7b98b0643910>,
 'conv5_block3_out': <keras.src.engine.functional.Functional at 0x7b98b0451bd0>}

In [15]:
class AuxFunctions:
    
    def __init__(self, _version=None, _type=None) -> None:
        self.__version = _version
        self.__type = _type
        self.__show = False
    
    def _showGraph(self, _show=False) -> None:
        self.__show = _show
        
    def _plot_metrics(self, _title=None, _kValues=None, _accuracies=None, _tag=None):
     
        plt.figure(figsize=(10, 6))
        
        plt.plot(_kValues, _accuracies, marker='o')
        sns.set(style="whitegrid")
        sns.lineplot(x=_kValues, y=_accuracies, marker='o')
        
        plt.title(f'{_title} vs. Number of Neighbors (k)')
        plt.xlabel('Number of Neighbors (k)')
        
        plt.ylabel(f'{_title}')
        plt.xticks(_kValues)
        plt.grid(True)
        
        dirPath = f'./graph/{self.__version}/{self.__version}_{self.__type}/{_tag}/'
        
        if(not os.path.isdir(dirPath)):
            os.makedirs(dirPath)
                
        plt.savefig(f'{dirPath}/{_title}.png')

        if(self.__show):
            plt.show()
        else:        
            plt.close()
        
    def _plot_confusion_matrix(self, _title=None, _yPred=None, _yTest=None, _nameOfFile=None,_tag=None):
        # Plot the confusion matrix 
        cm = confusion_matrix(_yTest, _yPred)

        display = ConfusionMatrixDisplay(confusion_matrix=cm)

        display.plot()

        plt.title(_title)
        
        dirPath = f'./matrix/{self.__version}/{self.__version}_{self.__type}/{_tag}/'
        
        if(not os.path.isdir(dirPath)):
            os.makedirs(dirPath)
                
        plt.savefig(f'{dirPath}/{_nameOfFile}.png')
            
        plt.close()
    

In [16]:
class Strategy(AuxFunctions):
    def __init__(self, _v=None, _type=None, _layer=None) -> None:
        super().__init__(_v, _type)
        self.__version = _v
        self.__type = _type
        self.__yPreds = []
        self.__losses = []
        self.__accuracies = []
        self.__layer = _layer
        self.__k_values = np.arange(1, 10)
        
        self.__dataset = []
        self.__labels = []
    
    def preProcessDataset(self, _model=None):
        # Load dataset 
        self.__loadDataset(_model)
        
    
    def __calculate_metrics(self, _model=None, _xTest=None, _yTest=None):
        # Calculate the accuracy
        y_pred = _model.predict(_xTest)
        acc = metrics.accuracy_score(_yTest, y_pred)

        # Calculate the loss
        y_pred_proba = _model.predict_proba(_xTest)
        loss = log_loss(_yTest, y_pred_proba)
        
        return acc, loss, y_pred
    
    def __loadDataset(self, _modelSelected=None):
        ## Usar texture para pegar o dataset de texturas

        dir = pathlib.Path(f"../../../Master-s_Thesis/database/images/{f'{self.__version}/{self.__version}_{self.__type}'}/")
        
        pastas = dir.glob("*")
        
        for subPasta in pastas:
            nomePasta = os.path.basename(subPasta)
            arquivos = subPasta.glob("*.*")
            
            for img in arquivos:
                aux = cv2.imread(str(img))
                aux = cv2.resize(aux,(224,224))
                aux = tf.expand_dims(aux,0)
                
                res = _modelSelected.predict(x = aux, verbose=0)
                
                self.__dataset.append(res[0])
                self.__labels.append(nomePasta)

        print("[+] Dataset Carregado!")
    
    def __encodeData(self):
        encoder = LabelEncoder()
        self.__labelEncoder = encoder.fit_transform(self.__labels)
    
    def __splitDataset(self):
        self.__X_train, self.__X_test, self.__y_train, self.__y_test = train_test_split(self.__dataset, self.__labelEncoder, test_size=0.3)
    
    def __trainWithKMeans(self):
        
        for k in self.__k_values:
            knn = KNeighborsClassifier(n_neighbors=k) 

            knn.fit(self.__X_train, self.__y_train)
            
            acc, loss, y_pred = self.__calculate_metrics(knn, self.__X_test, self.__y_test)
            
            self.__accuracies.append(acc)
            self.__losses.append(loss)
            self.__yPreds.append(y_pred) 
            
    def setLayer(self, _layer):
        self.__layer = _layer
        
    def training_V1(self):
        
        if(not self.__layer):
            print('Is necessary define a layer')
        else:        
            print(f'Strategy initialized {self.__version}_{self.__type} -> [{self.__layer}]')
            
            # Encode the training data
            self.__encodeData()
            print('[+] 1.2 Encode of training data finished')
            
            # Split of dataset
            self.__splitDataset()
            print('[+] 1.3 Split of dataset finished')
            
            # Training
            self.__trainWithKMeans()
            
            print('[+] 1.4 Training finished')
            
    def evaluate(self, _show=False):
        self._showGraph(_show)
        
        # Show graph of accuracy and loss
        self._plot_metrics(f'accuracy', self.__k_values, self.__accuracies, self.__layer)
        self._plot_metrics(f'loss', self.__k_values, self.__losses, self.__layer)
        
        # # Save confusion matrix
        for index, y in enumerate(self.__yPreds):
            self._plot_confusion_matrix(f'{self.__layer} - {index}', y, self.__y_test, f'k-{index}', self.__layer)

        # # Calculate final accuracy and final loss
        acc_final = self.__accuracies[len(self.__accuracies) - 1]
        
        loss_final = self.__losses[len(self.__losses) - 1]    

        print(f'\n[+] 1.5 Final Metrics: \n\n\t[Acc]: {"%.8f" %  acc_final}')
        
        print(f'\t[Loss]: {"%.8f" % loss_final}\n\n')

In [17]:
# Set version, dataset and layer 
t_data = 'v3'
t_version = '4'

In [18]:
for layer in layers_of_models:
    strategy = Strategy(t_data, t_version)
    
    # Preprocessing of data
    strategy.preProcessDataset(layers_of_models[layer])

    # Initialize training 
    # First we have to define the layer parameters
    strategy.setLayer(layer)

    # Second we need to training the model
    strategy.training_V1()

    # Last of the training, we need to evaluate the model
    strategy.evaluate()

[+] Dataset Carregado!
Strategy initialized v3_4 -> [conv1_conv]
[+] 1.2 Encode of training data finished
[+] 1.3 Split of dataset finished
[+] 1.4 Training finished

[+] 1.5 Final Metrics: 

	[Acc]: 0.67980296
	[Loss]: 2.91508101


[+] Dataset Carregado!
Strategy initialized v3_4 -> [conv2_block3_out]
[+] 1.2 Encode of training data finished
[+] 1.3 Split of dataset finished
[+] 1.4 Training finished

[+] 1.5 Final Metrics: 

	[Acc]: 0.64532020
	[Loss]: 2.47842435


[+] Dataset Carregado!
Strategy initialized v3_4 -> [conv3_block4_out]
[+] 1.2 Encode of training data finished
[+] 1.3 Split of dataset finished
[+] 1.4 Training finished

[+] 1.5 Final Metrics: 

	[Acc]: 0.74876847
	[Loss]: 1.00195618


[+] Dataset Carregado!
Strategy initialized v3_4 -> [conv4_block6_out]
[+] 1.2 Encode of training data finished
[+] 1.3 Split of dataset finished
[+] 1.4 Training finished

[+] 1.5 Final Metrics: 

	[Acc]: 0.84236453
	[Loss]: 1.08465773


[+] Dataset Carregado!
Strategy initialized v3_4 -

In [19]:
# knn = KNeighborsClassifier(n_neighbors=5)

# resultado = cross_validation(knn,dados,labels_encoded)

# print(resultado)